In [ ]:
import sys
sys.path.append('../../libsigma')
import geopandas as gpd
from osgeo import gdal
from shapely.geometry import box
from classification import rasterization, get_samples_from_roi
import os

# --- Paramètres ---
BD_FORET_PATH = "../../data/project/FORMATION_VEGETALE.shp"  # Chemin vers la BD Forêt
RASTER_S2_PATH = "../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif"  # Chemin vers le raster Sentinel-2
RASTERIZED_VECTOR_PATH = "../results/data/sample/rasterized_vector.tif"  # Rasterisation temporaire
OUTPUT_SAMPLES_PATH = "../results/data/sample/Sample_BD_foret_T31TCJ.shp"  # Chemin pour sauvegarder les échantillons

# Liste des codes TFV d'intérêt
CODE_TFV_VALUES = [
    "Forêt fermée d’un autre feuillu pur",
    "Forêt fermée de châtaignier pur",
    "Forêt fermée de hêtre pur",
    "Forêt fermée de chênes décidus purs",
    "Forêt fermée de robinier pur",
    "Peuplerai",
    "Forêt fermée à mélange de feuillus",
    "Forêt fermée de feuillus purs en îlots",
    "Forêt fermée d’un autre conifère pur autre que pin",
    "Forêt fermée de mélèze pur",
    "Forêt fermée de sapin ou épicéa",
    "Forêt fermée à mélange d’autres conifères",
    "Forêt fermée d’un autre pin pur",
    "Forêt fermée de pin sylvestre pur",
    "Forêt fermée à mélange de pins purs",
    "Forêt fermée de douglas pur",
    "Forêt fermée de pin laricio ou pin noir pur",
    "Forêt fermée de pin maritime pur",
    "Forêt fermée à mélange de conifères",
    "Forêt fermée de conifères en îlots",
    "Forêt fermée à mélange de conifères prépondérants et feuillus",
    "Forêt fermée à mélange de feuillus prépondérants et conifères"
]

# --- Fonction principale ---
def rasterize_and_sample(bd_foret_path, raster_s2_path, rasterized_vector_path, output_samples_path, code_tfv_values):
    # Charger la BD Forêt
    bd_foret = gpd.read_file(bd_foret_path)
    print("BD Forêt chargée")

    # Filtrer par les valeurs de Code TFV
    bd_foret_filtered = bd_foret[bd_foret['TFV'].isin(code_tfv_values)]
    print(f"Nombre de polygones après filtrage des codes TFV : {len(bd_foret_filtered)}")

    # Sauvegarder le vecteur filtré temporairement
    filtered_vector_path = "../results/data/sample/filtered_bd_foret.shp"
    bd_foret_filtered.to_file(filtered_vector_path)
    print("Vecteur filtré sauvegardé temporairement")

    # Rasteriser le vecteur filtré en utilisant la fonction rasterization
    rasterization(filtered_vector_path, raster_s2_path, rasterized_vector_path, field_name="TFV")
    print("Rasterisation terminée")

    # Extraire les échantillons avec la fonction get_samples_from_roi
    samples, labels, coordinates = get_samples_from_roi(raster_s2_path, rasterized_vector_path)
    print(f"Nombre d'échantillons extraits : {samples.shape[0]}")

    # Sauvegarder les échantillons
    np.save(output_samples_path, samples)
    print(f"Échantillons sauvegardés dans : {output_samples_path}")

    # Nettoyage des fichiers temporaires
    os.remove(filtered_vector_path)
    os.remove(rasterized_vector_path)
    print("Fichiers temporaires supprimés")

# --- Exécution du code ---
rasterize_and_sample(BD_FORET_PATH, RASTER_S2_PATH, RASTERIZED_VECTOR_PATH, OUTPUT_SAMPLES_PATH, CODE_TFV_VALUES)


BD Forêt chargée
Nombre de polygones après filtrage des codes TFV : 17132
Vecteur filtré sauvegardé temporairement
Rasterisation en cours...
otbcli_Rasterization -in ../results/data/sample/filtered_bd_foret.shp -im ../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif -out ../results/data/sample/rasterized_vector.tif -mode attribute -mode.attribute.field TFV


CalledProcessError: Command 'otbcli_Rasterization -in ../results/data/sample/filtered_bd_foret.shp -im ../../data/images/SENTINEL2B_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B2.tif -out ../results/data/sample/rasterized_vector.tif -mode attribute -mode.attribute.field TFV' returned non-zero exit status 127.